In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
import re
import copy

In [10]:
Placeholder=None#To handle missing values when scraping
Headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0'}

In [11]:
movie_name=[]
movie_year=[]
movie_rating=[]
movie_votes=[]
movie_metascore=[]
Gross_income=[]
Movie_Runtime=[]
Genre=[]
Age_rated=[]
Directors=[]
Stars=[]
description=[]

In [22]:
import time
start=time.time()
pages=np.arange(1,100,50)
for page in pages:
    url=f'https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&start={str(page)}'
    try:
        webpage = requests.get(url, headers=Headers).text
    except requests.exceptions.ConnectionError:
        print(f"Connection error occurred. Retrying after 5 sec")
        time.sleep(5)  # Wait for 5 seconds before retrying
        continue  # Skip the rest of the loop and move to the next page

    # webpage=requests.get(url,headers=Headers).text
    soup=BeautifulSoup(webpage,'lxml')

    movie_data=soup.find_all("h3",class_="ipc-title__text")
    # print(movie_data.find('h3', class_='ipc-title__text'))
    print(movie_data)
    for movies in movie_data:
        # print(movies)
        soup.prettify()
        #Movie name
        movie_name.append(movies.h3.a.text.strip())

        #Years
        years=movies.h3.find("span",class_='lister-item-year text-muted unbold').text.strip("()")
        movie_year.append(years)



        #Rating
        rate=movies.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
        movie_rating.append(rate)

        #Meta Score
        meta = movies.find('span', class_ = 'metascore').text.replace(' ', '') if movies.find('span', class_ = 'metascore') else Placeholder
        movie_metascore.append(meta)

        #Gross income and Votes

        value = movies.find_all('span', attrs = {'name': "nv"})

        vote = value[0].text
        gross = value[1].text if len(value)>1 else Placeholder#handling missing value
        Gross_income.append(gross)
        movie_votes.append(vote)


        #Runtime,Genre and Certificate

        runtime_span = movies.find("span", class_="runtime")
        genre_span = movies.find("span", class_="genre")
        certificate_span = movies.find("span", class_="certificate")

            # Handle missing runtime information
        if runtime_span:
            runtime_text = runtime_span.text.strip('min')
        else:
            runtime_text = Placeholder

            # Handle missing genre information
        if genre_span:
            genre_text = genre_span.text.strip().split(',')
        else:
            genre_text = [Placeholder]

             # Handle missing certificate information
        if certificate_span:
            certificate_text = certificate_span.text.strip()
        else:
            certificate_text = Placeholder


        #Description

        Movie_Runtime.append(runtime_text)
        Genre.append(genre_text)
        Age_rated.append(certificate_text)

        describe = movies.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else Placeholder
        description.append(description_)


        #Director and Stars using Regex
        Cast = movies.find('p', class_='').text.strip().replace('\n', '')

              # Extract directors and stars using regex patterns
        directors_pattern = r"Directors?:(.*?)(?:\||Stars:|$)"
        stars_pattern = r"Stars:(.*?)(?:\||$)"

              # Extract directors using regex
        directors_match = re.search(directors_pattern, Cast, re.DOTALL)
        directors = [director.strip() for director in directors_match.group(1).split(',')] if directors_match else []

              # Extract stars using regex
        stars_match = re.search(stars_pattern, Cast, re.DOTALL)
        stars = [star.strip() for star in stars_match.group(1).split(',')] if stars_match else []

        Directors.append(directors)
        Stars.append(stars)

end=time.time()
total=end-start
print(total)

[<h3 class="ipc-title__text">1. The Silence of Swastika</h3>, <h3 class="ipc-title__text">2. Nhà tù Shawshank</h3>, <h3 class="ipc-title__text">3. Bố Già</h3>, <h3 class="ipc-title__text">4. Ramayana: The Legend of Prince Rama</h3>, <h3 class="ipc-title__text">5. Hababam Sinifi</h3>, <h3 class="ipc-title__text">6. CM101MMXI Fundamentals</h3>, <h3 class="ipc-title__text">7. Kỵ Sĩ Bóng Đêm</h3>, <h3 class="ipc-title__text">8. Bản Danh Sách Của Schindler</h3>, <h3 class="ipc-title__text">9. Chúa Tể Của Những Chiếc Nhẫn: Sự Quay Trở Lại Của Nhà Vua</h3>, <h3 class="ipc-title__text">10. Bố Già Phần II</h3>, <h3 class="ipc-title__text">11. 12 Người Đàn Ông Giận Dữ</h3>, <h3 class="ipc-title__text">12. Aynabaji</h3>, <h3 class="ipc-title__text">13. Chuyện Tào Lao</h3>, <h3 class="ipc-title__text">14. Thanh Gươm Diệt Quỷ: Phần Kỹ Viện Trấn</h3>, <h3 class="ipc-title__text">15. David Attenborough: Một Cuộc Đời Trên Trái Đất</h3>, <h3 class="ipc-title__text">16. O.J.: Made in America</h3>, <h3 c

In [5]:
Dataset = pd.DataFrame({
    "Movie Name": movie_name, "Year of Release" : movie_year, "Run Time in minutes": Movie_Runtime,"Movie Rating": movie_rating,"Votes":movie_votes,"MetaScore":movie_metascore,
    "Gross":Gross_income,"Genre":Genre,"Certification":Age_rated,"Director":Directors,"Stars":Stars,"Description":description
})


In [13]:
movie_name

[]

In [6]:
data = pd.DataFrame(columns = Dataset.columns, data = copy.deepcopy(Dataset.values))

In [7]:
data

,Movie Name,Year of Release,Run Time in minutes,Movie Rating,Votes,MetaScore,Gross,Genre,Certification,Director,Stars,Description


In [311]:
data['Year of Release']=data['Year of Release'].str.replace(r'\D', '').astype(int)#Removes all the unwanted values with regex \D  and changes them into int
data.Votes=data.Votes.str.replace(',','').astype(int)
data['Movie Rating']=data['Movie Rating'].astype(float)
data['Run Time in minutes']=data['Run Time in minutes'].astype(int)

for index,value in enumerate(data.Gross):
  if value!=None:
    data.Gross[index]=value.replace('$','').replace('M','')
data.Gross=pd.to_numeric(data.Gross)
data.Gross=data.Gross*1000000

data.MetaScore=pd.to_numeric(data.MetaScore)


<ipython-input-311-497fe3c909c0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Year of Release']=data['Year of Release'].str.replace(r'\D', '').astype(int)#Removes all the unwanted values with regex \D  and changes them into int
<ipython-input-311-497fe3c909c0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Gross[index]=value.replace('$','').replace('M','')


In [312]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Movie Name           10000 non-null  object 
 1   Year of Release      10000 non-null  int64  
 2   Run Time in minutes  10000 non-null  int64  
 3   Movie Rating         10000 non-null  float64
 4   Votes                10000 non-null  int64  
 5   MetaScore            7974 non-null   float64
 6   Gross                7085 non-null   float64
 7   Genre                10000 non-null  object 
 8   Certification        9631 non-null   object 
 9   Director             10000 non-null  object 
 10  Stars                10000 non-null  object 
 11  Description          10000 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 937.6+ KB


In [314]:
data['Description']=data['Description'].apply(lambda x:x.split())

In [315]:
data

,Movie Name,Year of Release,Run Time in minutes,Movie Rating,Votes,MetaScore,Gross,Genre,Certification,Director,Stars,Description
0,The Shawshank Redemption,1994,142,9.3,2804443,82.0,28340000.0,[Drama],R,[Frank Darabont],"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...","[Over, the, course, of, several, years,, two, ..."
1,The Godfather,1972,175,9.2,1954174,100.0,134970000.0,"[Crime, Drama]",R,[Francis Ford Coppola],"[Marlon Brando, Al Pacino, James Caan, Diane K...","[Don, Vito, Corleone,, head, of, a, mafia, fam..."
2,Ramayana: The Legend of Prince Rama,1993,135,9.2,12995,NaN,NaN,"[Animation, Action, Adventure]",PG,"[Ram Mohan, Yûgô Sakô, Koichi Saski]","[Arun Govil, Nikhil Kapoor, Edie Mirman, Rael ...","[An, anime, adaptation, of, the, Hindu, epic, ..."
3,The Chaos Class,1975,87,9.2,42231,NaN,NaN,"[Comedy, Drama]",None,[Ertem Egilmez],"[Kemal Sunal, Münir Özkul, Halit Akçatepe, Tar...","[Lazy,, uneducated, students, share, a, very, ..."
4,The Dark Knight,2008,152,9.0,2786129,84.0,534860000.0,"[Action, Crime, Drama]",PG-13,[Christopher Nolan],"[Christian Bale, Heath Ledger, Aaron Eckhart, ...","[When, the, menace, known, as, the, Joker, wre..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Golmaal Again,2017,140,4.9,10183,NaN,1010000.0,"[Action, Comedy, Fantasy]",Not Rated,[Rohit Shetty],"[Ajay Devgn, Arshad Warsi, Tabu, Shreyas Talpade]","[The, gang, encounters, with, some, spiritual,..."
9996,Not to Forget,2021,84,4.9,11191,NaN,NaN,[Drama],None,[Valerio Zanoli],"[Karen Grassle, Louis Gossett Jr., Tatum O'Nea...","[After, a, lifetime, of, scams,, a, self-cente..."
9997,Housefull 3,2016,140,4.9,10308,NaN,1160000.0,"[Action, Comedy, Romance]",Not Rated,"[Sajid, Farhad Samji]","[Akshay Kumar, Abhishek Bachchan, Riteish Desh...","[A, father, doesn't, want, his, three, daughte..."
9998,A.I. Rising,2018,85,4.9,11187,NaN,NaN,"[Drama, Romance, Sci-Fi]",R,[Lazar Bodroza],"[Sebastian Cavazza, Stoya, Marusa Majer, Kirst...","[An, intimate, relationship, between, a, human..."


In [316]:
data.to_csv("Movies_IMDB.csv")